In [1]:
ip='172.31.19.202'
db='amazon_sp_api'
password='Techblooprint123'
user="blooprint"
from pyspark.sql import SparkSession
import os
from clickhouse_driver import Client
import datetime
from pyspark.sql.types import FloatType
from functools import reduce
# spark = SparkSession.builder \
#     .appName("BigCSVWrite") \
#     .config("spark.driver.memory", "5g") \
#     .config("spark.executor.memory", "6g") \
#     .getOrCreate()
# spark.conf.set("spark.sql.adaptive.enabled", "true")
# spark.conf.set("spark.sql.shuffle.partitions", "50")
spark = (
    SparkSession.builder
    .appName("BigCSVWrite")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.memory", "8g")
    .config("spark.executor.memoryOverhead", "2g")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.adaptive.skewJoin.enabled", "true")
    .config("spark.sql.shuffle.partitions", "400")
    .config("spark.default.parallelism", "400")
    .config("spark.memory.fraction", "0.5")
    .config("spark.memory.storageFraction", "0.3")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size", "2g")
    .getOrCreate()
)

from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F
import pandas as pd
# Should not raise:
spark.sparkContext._jvm.java.lang.Class.forName("org.postgresql.Driver")

URL  = os.getenv("PG_JDBC_URL", f"jdbc:postgresql://{ip}:5433/{db}")
USER = os.getenv("PG_USER", user)
PWD  = os.getenv("PG_PASSWORD", password)
import pygsheets
import os

def read_sheet(spreadsheet_id,sheet_name,index=0):
    try:
        index=int(index)
        index=index%12
        service_file_path = os.path.join(os.getcwd(), f"key{index}.json")
        gc = pygsheets.authorize(service_file=service_file_path)
        sh = gc.open_by_key(spreadsheet_id)
        selected_sheet = sh.worksheet_by_title(sheet_name)
        df = selected_sheet.get_as_df()
        # database.rename_columns(df)
        return df
    except Exception as e:
        print(f"jhqwdf",e)
        pass
base_sheet='1_zTZSaaAM0svrV26XoX59ZcTJ6MMcpSNV64UOvRDHeA'
df=read_sheet(base_sheet,'AZ',2)
all_sheet_ids=df['sheet_id'].to_list()
all_target_df=[]
for sheet_id in all_sheet_ids:
    print(sheet_id)
    target_df=read_sheet(sheet_id,'Target ACOS AZ')
    if target_df is not None:
        target_df=target_df[['campaign_name','Target ACOS']].drop_duplicates()
        all_target_df.append(target_df)

all_campaign_target=pd.concat(all_target_df)
all_campaign_target.to_csv("all_camp_target.csv",index=False)
# 2. Read CSV file
target_df = spark.read.csv(
    "all_camp_target.csv",
    header=True,      # use first row as column names
    inferSchema=True  # infer data types automatically
)



(
    target_df.write
    .format("jdbc")
    .option("url", URL)
    .option("user", USER)
    .option("password", PWD)
    .option("driver", "org.postgresql.Driver")
    .option("dbtable", "public.target_info")  # destination table
    .mode("overwrite")  # creates table if not exists, replaces if exists
    .save()
)
os.remove('all_camp_target.csv')


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 00:34:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1VHNcEG4EExsROyhb4eTU0yl4JYktNOrBW9VeUCU25cw


/home/jovyan/.local/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


1Qkb_jo6DkEFlL8lnP2b3IVgUf8H2ww21TaoQgNTSJAE


/home/jovyan/.local/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


1CxhjF6Rz4dg6udRjMCVehBk2cKO2B-U46SAmqmcgCyI
1R1yAkW0C1NC_H2512AEI15BooNORv7Ad6Bs5EAdFRTE


/home/jovyan/.local/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


15fn4kLCOU0YRT6ggyPX8s170-UQOFmEXpEmAln-OKCI
1CiWc0CHuf8WtEOKg_nVRRijtNBRb60ardmVcgUZsp9A
1fL0F1CQgkRGWW_iu0pY3cf5SOFQFw_ZbAqyDmvWRpgY
jhqwdf 
1nv43vtcK6tNlXxmHDj2dySt646i4jG1JyyiTz6b8hUo


In [2]:
portfolio_query = "(SELECT * FROM public.portfolio_id_name_mapping) AS p"

portfolio_id_name_mapping = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", portfolio_query)
      .load())
campaign_query = "(SELECT * FROM public.campaigns) as c"
campaigns = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", campaign_query)
      .load())
competitor_query = "(SELECT * FROM public.competitor_brand_data) as cb"
competitors = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", competitor_query)
      .load())
sb_kw_query = "(SELECT * FROM public.sponsored_brands_search_term) as sb"
sb_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sb_kw_query)
      .load())
sp_kw_query = "(SELECT * FROM public.sponsored_products_search_term) as sp"
sp_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sp_kw_query)
      .load())
sd_kw_query = "(SELECT * FROM public.sponsored_display_targeting) as sd"
sd_kw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", sd_kw_query)
      .load())

targets_query = "(SELECT * FROM public.targets ) as t"
targets = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", targets_query)
      .load())
bid_query="""(SELECT unnamed_subquery.keyword_id,
            unnamed_subquery.bid,
            unnamed_subquery.bid_tym
           FROM ( SELECT bid_values.keyword_id,
                    bid_values.bid,
                    bid_values.updated_date::date AS bid_tym,
                    rank() OVER (PARTITION BY bid_values.keyword_id ORDER BY bid_values.updated_date DESC) AS rank
                   FROM bid_values) unnamed_subquery
          WHERE unnamed_subquery.rank = 1 ) bid"""
bid_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", bid_query)
      .load())
ads_query = "(SELECT * FROM public.ads ) as a"
ads_df_raw = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", ads_query)
      .load())

campaign_df=campaigns.select(F.col("campaignId").alias("campaign_id"),
                 F.col("portfolioId").alias("portfolioId_camp"),
                 F.col("name").alias("campaign_name"),
                 ).dropDuplicates(['campaign_id'])

portfolio_df=portfolio_id_name_mapping.select(F.col("portfolioId").alias("portfolioId"),
                                 F.col("name").alias("portfolio_name")).dropDuplicates(['portfolioId'])



portfolio_mapping=campaign_df.join(portfolio_df,on=campaign_df['portfolioId_camp']==portfolio_df['portfolioId'],how='left')

sb_search_term=sb_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("keywordId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("keywordText").alias("keyword"),
             F.col("searchTerm").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales").alias("ads_sale"),
             F.col("unitsSold").alias("ads_units"),
             F.col("matchType").alias("match_type"),



             ).withColumn("kt_type",F.lit("keyword_id"))
sd_search_term=sd_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("targetingId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("targetingText").alias("keyword"),
             F.lit("not_applicable").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales").alias("ads_sale"),
             F.col("unitsSold").alias("ads_units"),
             F.lit("DSP TGT").alias("match_type"),

             ).withColumn("kt_type",F.lit("targeting_id"))
sp_search_term=sp_kw.select(F.col("account_name"),
             F.split(F.col("campaignId").cast("string"),"\\.").getItem(0).alias("campaignId"),
             F.col("campaignName").alias("campaign_name"),
             F.split(F.col("adGroupId").cast("string"),"\\.").getItem(0).alias("ad_group_id"),
             F.col("adGroupName").alias("ad_group_name"),
             F.split(F.col("keywordId").cast("string"),"\\.").getItem(0).alias("keyword_id"),
             F.col("targeting").alias("keyword"),
             F.col("searchTerm").alias("search_term"),
             F.col("date").cast("date").alias("date"),
             F.col("impressions").alias("impressions"),
             F.col("clicks").alias("clicks"),
             F.col("cost").alias("ad_spend"),
             F.col("sales30d").alias("ads_sale"),
             F.col("unitsSoldSameSku1d").alias("ads_units"),
             F.col("matchType").alias("match_type"),



             ).withColumn("kt_type",F.lit("targeting_id"))
             
keyword_df=sp_search_term.unionByName(sb_search_term).unionByName(sd_search_term)
keyword_df=keyword_df.withColumn(
    "new_match_type",
    F.when(
        (F.col("match_type") == "TARGETING_EXPRESSION") & F.col("keyword").like("%asin=%"),
        F.lit("PT EXACT")
    ).when(
        (F.col("match_type") == "TARGETING_EXPRESSION") & F.col("keyword").like("%category=%"),
        F.lit("PT CAT")
    ).when(
        (F.col("match_type") == "TARGETING_EXPRESSION") & F.col("keyword").like("%asin-expanded=%"),
        F.lit("PT EXP")
    ).when(
        F.col("match_type") == "TARGETING_EXPRESSION_PREDEFINED",
        F.upper(F.col("keyword"))
    ).otherwise(F.col("match_type"))
)


target_df = (
    targets
    .select(
        F.col("campaignId").alias("campaign_id"),
        F.col("targetType").alias("targeting_type"),
        F.col("targetDetails_event").alias("target_event")
    )
    .withColumn(
        "target_group",
        F.when(F.upper(F.col("targeting_type")) == "PRODUCT", "PT")
         .when(F.upper(F.col("targeting_type")) == "KEYWORD", "KT")
         .when(F.upper(F.col("targeting_type")).isin("AUTO", "THEME"), "AT")
         .when(F.upper(F.col("targeting_type")) == "PRODUCT_CATEGORY", "CT")
         .when(F.upper(F.col("targeting_type")) == "AUDIENCE", "ADT")
         .when(
             (F.upper(F.col("targeting_type")).isin("PRODUCT_CATEGORY_AUDIENCE", "PRODUCT_AUDIENCE")) &
             (F.upper(F.col("target_event")) == "PURCHASES"),
             "PR"
         )
         .when(
             (F.upper(F.col("targeting_type")).isin("PRODUCT_CATEGORY_AUDIENCE", "PRODUCT_AUDIENCE")) &
             (F.upper(F.col("target_event")) == "VIEWS"),
             "VR"
         )
         .otherwise("Not Applicable")
    )
    .dropDuplicates()
    .select("campaign_id","target_group")
)
target_df = target_df.dropDuplicates(["campaign_id"])
bid_df = bid_df.dropDuplicates(["keyword_id"])
keyword_df=keyword_df.join(
    F.broadcast(bid_df),
    on="keyword_id",
    how="left"
)
portfolio_mapping=portfolio_mapping.join(target_df,on='campaign_id',how='left').withColumnRenamed("target_group","targeting_type")


ads_df=(ads_df_raw.select(
    F.col("account_name").alias("ads_account_name"),
    F.col("campaignId").alias("ads_campaign_id"),
    F.col("adProduct").alias("ad_product"),
    F.col("adType").alias("ad_type"),
    F.col("state").alias("CAMP STATUS"),
    F.col("creative_products")
).dropDuplicates()
.withColumn("ads_type",
    F.when(F.col("ad_product")=="SPONSORED_PRODUCTS","SP")
    .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type").isin("BRAND_VIDEO","VIDEO")),"SBV")
    .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type")=="PRODUCT_COLLECTION"),"SBB")
    .when((F.col("ad_product")=="SPONSORED_BRANDS")&(F.col("ad_type")=="STORE_SPOTLIGHT"),"SBS")
    .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="IMAGE"),"SDI")
    .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="PRODUCT_AD"),"SDP")
    .when((F.col("ad_product")=="SPONSORED_DISPLAY")&(F.col("ad_type")=="VIDEO"),"SDV")
    .otherwise("Not Applicable")          
).select(
    "ads_account_name","ads_campaign_id","ads_type","CAMP STATUS","creative_products"
)
)

ads_df = ads_df.dropDuplicates(["ads_campaign_id"])
keyword_df=keyword_df.join(
    F.broadcast(ads_df),
    (keyword_df["account_name"] == ads_df["ads_account_name"]) &
    (keyword_df["campaignId"] == ads_df["ads_campaign_id"]),
    how="inner"
).drop("ads_account_name","ads_campaign_id")


kw_meta_data=keyword_df.select(
    "account_name","ads_type","campaignId","ad_group_id","ad_group_name","kt_type","keyword_id","keyword","new_match_type","bid","bid_tym","creative_products"
).dropDuplicates()
keyword_df=keyword_df.groupby("account_name","campaignId","ad_group_id","ad_group_name","kt_type","keyword_id","keyword","new_match_type","date").agg(
F.sum("impressions").alias("impressions"),
F.sum("clicks").alias("clicks"),
F.sum("ad_spend").alias("ad_spend"),
F.sum("ads_sale").alias("ads_sale"),
F.sum("ads_units").alias("ads_units")
)
keyword_df=keyword_df.withColumn("month",
    F.date_format(F.col("date"),'MMM')
).withColumn("year",
    F.year(F.col("date"))
)
today = datetime.datetime.today() - datetime.timedelta(days = 0)
yesterday = datetime.datetime.today() - datetime.timedelta(days = 1)
last_7_days= datetime.datetime.today() - datetime.timedelta(days = 7)
last_14_days= datetime.datetime.today() - datetime.timedelta(days = 14)
last_30_days= datetime.datetime.today() - datetime.timedelta(days = 30)
last_60_days= datetime.datetime.today() - datetime.timedelta(days = 60)
today=today.strftime('%Y-%m-%d')
yesterday = yesterday.strftime('%Y-%m-%d')
last_7_days = last_7_days.strftime('%Y-%m-%d')
last_14_days = last_14_days.strftime('%Y-%m-%d')
last_30_days = last_30_days.strftime('%Y-%m-%d')
last_60_days = last_60_days.strftime('%Y-%m-%d')
# keyword_df=keyword_df.filter(F.col("date") != today)


portfolio_meta_data=kw_meta_data.join(F.broadcast(portfolio_mapping),how='left',on=kw_meta_data['campaignId']==portfolio_mapping['campaign_id'])
portfolio_meta_data=portfolio_meta_data.drop('campaign_id','portfolioId_camp')
portfolio_meta_data=portfolio_meta_data.dropDuplicates(["account_name","campaignId","ad_group_id","keyword_id"])


def calculate_metrics(i, mode=1):
    if mode == 1:
        if i == last_14_days:
            d = 'last_14_days'
        elif i == last_7_days:
            d = 'last_7_days'
        elif i == yesterday:
            d = 'yesterday'
        elif i == last_30_days:
            d = 'last_30_days'
        elif i == last_60_days:
            d = 'last_60_days'
        elif i == today:
            d = 'today'
            base_df = keyword_df.filter(F.col("date") == today)

        if i == yesterday:
            base_df = keyword_df.filter(F.col("date") == yesterday)
        else:
            base_df = keyword_df


        temp_df = base_df.filter(F.col("date") >= i)
    else:
        month = i[0]
        year = i[1]
        d = month

        temp_df = keyword_df.filter(
            (F.col('month') == month) & (F.col('year') == year)
        )

    agg_df = temp_df.groupby("account_name", "campaignId", "ad_group_id", "keyword_id").agg(
        F.round(F.sum("impressions"), 0).alias(f"impressions_{d}"),
        F.round(F.sum("clicks"), 0).alias(f"clicks_{d}"),
        F.round(F.sum("ad_spend"), 0).alias(f"ad_spend_{d}"),
        F.round(F.sum("ads_sale"), 0).alias(f"ads_sale_{d}"),
        F.round(F.sum("ads_units"), 0).alias(f"ads_units_{d}")
    )

    agg_df = (
        agg_df
        .withColumn(
            f"CPC_{d}",
            F.when(F.col(f"clicks_{d}") != 0,
                   F.round(F.col(f"ad_spend_{d}") / F.col(f"clicks_{d}"), 2))
            .otherwise(F.lit(0))
        )
        .withColumn(
            f"CPA_{d}",
            F.when(F.col(f"ads_units_{d}") != 0,
                   F.round(F.col(f"ad_spend_{d}") / F.col(f"ads_units_{d}"), 2))
            .otherwise(F.lit(0))
        )
        .withColumn(
            f"ACOS_{d}",
            F.when(F.col(f"ads_sale_{d}") != 0,
                   F.round((F.col(f"ad_spend_{d}") * 100) / F.col(f"ads_sale_{d}"), 2))
            .otherwise(F.lit(0))
        )
        .withColumn(
            f"CVR_{d}",
            F.when(F.col(f"clicks_{d}") != 0,
                   F.round((F.col(f"ads_units_{d}") * 100) / F.col(f"clicks_{d}"), 2))
            .otherwise(F.lit(0))
        )
        .withColumn(
            f"CTR_{d}",
            F.when(F.col(f"impressions_{d}") != 0,
                   F.round((F.col(f"clicks_{d}") * 100) / F.col(f"impressions_{d}"), 2))
            .otherwise(F.lit(0))
        )
    )

    # ✅ Replace nulls in numeric columns with 0
    agg_df = agg_df.fillna(0)


    return agg_df

current_month = datetime.date.today().month


dfs = []
time_cut=[today,yesterday,last_7_days,last_14_days,last_30_days,last_60_days]
for time_duration in time_cut:
    df=calculate_metrics(time_duration, 1)
    dfs.append(df)


for m in range(current_month ,current_month - 4,-1):
    month_name = datetime.date(1900, m, 1).strftime("%b")

    df = calculate_metrics([month_name, 2025], 2)
    
    dfs.append(df)



if dfs:
    final_df = reduce(
        lambda left, right: left.join(
            right,
            on=["account_name", "campaignId", "ad_group_id", "keyword_id"],
            how="outer"
        ),
        dfs
    )

final_df=final_df.fillna(0)


In [3]:

campaign_target_query = "(SELECT * FROM public.target_info) AS p"

campaign_target_df = (spark.read.format("jdbc")
      .option("url", URL)
      .option("user", USER)
      .option("password", PWD)
      .option("driver", "org.postgresql.Driver")
      .option("dbtable", campaign_target_query)
      .load())
# Keys to join on
final_df = (
    final_df
    .withColumnRenamed("account_name", "account_name_kw")

    .withColumnRenamed("campaignId", "campaignId_kw")
    .withColumnRenamed("ad_group_id", "ad_group_id_kw")
    .withColumnRenamed("keyword_id", "keyword_id_kw")
)

condition = (
    (final_df['account_name_kw'] == portfolio_meta_data['account_name'])&
    (final_df['campaignId_kw'] == portfolio_meta_data['campaignId']) &
    (final_df['ad_group_id_kw'] == portfolio_meta_data['ad_group_id']) &
    (final_df['keyword_id_kw'] == portfolio_meta_data['keyword_id'])
)

final_mapped_data = (
    final_df
    .join(
        F.broadcast(portfolio_meta_data),
        on=condition,
        how="left"   # keeps all rows from final_df
    )
)

mapped_acos=(
    final_mapped_data
    .join(
        F.broadcast(campaign_target_df),
        on='campaign_name',   # join on multiple columns
        how="left"      # keep all rows from final_df
    )
)
ads_automation_sheet='14aA8eKlsJqCd8mrNZcZOcnwW5pGICTZKs7DVE-IjO8Q'
df=read_sheet(ads_automation_sheet,'Account_level_setting',2)
df.to_csv("account_setting.csv",index=False)
account_setting=spark.read.csv(
    "account_setting.csv",
    header=True,      # use first row as column names
    inferSchema=True  # infer data types automatically
)
account_setting_joined = mapped_acos.alias("m").join(
    account_setting.alias("a"),
    on="account_name",
    how="left"
)

mapped_acos_ratio = (
    account_setting_joined
    .withColumn(
        "target_acos_new",
        F.coalesce(F.col("Target ACOS"), F.col("target_acos"))
    )
    .drop("Target ACOS", "target_acos")
    .withColumnRenamed("target_acos_new", "target_acos")
    .withColumn("target_acos", F.col("target_acos").cast(FloatType()))
)
df=mapped_acos_ratio
from pyspark.sql import functions as F, Window
w = Window.partitionBy("portfolio_name","keyword_id").orderBy(F.col("ads_sale_last_60_days").desc())
w_unordered = Window.partitionBy("portfolio_name")
df = df.withColumn("cumulative_sales", F.sum("ads_sale_last_60_days").over(w))
df = df.withColumn("cumulative_sales", F.sum("ads_sale_last_60_days").over(w))

# total sales per portfolio
df = df.withColumn("total_sales_portfolio", F.sum("ads_sale_last_60_days").over(w_unordered))

# cumulative percentage
df = df.withColumn("cumulative_percent", F.col("cumulative_sales") / F.col("total_sales_portfolio"))
df = df.withColumn(
    "priority_index",
    F.when(F.col("ads_sale_last_60_days") == 0, F.lit(-1)) # if sales = 0
        .when(F.col("cumulative_percent") <= 0.8, F.lit(1))   # top 80%
        .otherwise(F.lit(0))                                  # rest
)
all_columns=df.columns
columns_adjusted = [
    'account_name',
    'portfolioId',
    'portfolio_name',
    'ads_type',

    'campaignId',
    'campaign_name',
    'ad_group_id',
    'ad_group_name',
    'kt_type',
    'keyword_id',
    'keyword',
    'new_match_type',
    'targeting_type', 'bid', 'bid_tym',
    'creative_products',
    'target_acos',
    'priority_index',

    # ---- Daily Metrics ----
    'impressions_today',
    'clicks_today',
    'ad_spend_today',
    'ads_sale_today',
    'ads_units_today',
    'CPC_today',
    'CPA_today',
    'ACOS_today',
    'CVR_today',
    'CTR_today',

    'impressions_yesterday',
    'clicks_yesterday',
    'ad_spend_yesterday',
    'ads_sale_yesterday',
    'ads_units_yesterday',
    'CPC_yesterday',
    'CPA_yesterday',
    'ACOS_yesterday',
    'CVR_yesterday',
    'CTR_yesterday',

    # ---- Rolling Windows ----
    'impressions_last_7_days',
    'clicks_last_7_days',
    'ad_spend_last_7_days',
    'ads_sale_last_7_days',
    'ads_units_last_7_days',
    'CPC_last_7_days',
    'CPA_last_7_days',
    'ACOS_last_7_days',
    'CVR_last_7_days',
    'CTR_last_7_days',

    'impressions_last_14_days',
    'clicks_last_14_days',
    'ad_spend_last_14_days',
    'ads_sale_last_14_days',
    'ads_units_last_14_days',
    'CPC_last_14_days',
    'CPA_last_14_days',
    'ACOS_last_14_days',
    'CVR_last_14_days',
    'CTR_last_14_days',

    'impressions_last_30_days',
    'clicks_last_30_days',
    'ad_spend_last_30_days',
    'ads_sale_last_30_days',
    'ads_units_last_30_days',
    'CPC_last_30_days',
    'CPA_last_30_days',
    'ACOS_last_30_days',
    'CVR_last_30_days',
    'CTR_last_30_days',

    'impressions_last_60_days',
    'clicks_last_60_days',
    'ad_spend_last_60_days',
    'ads_sale_last_60_days',
    'ads_units_last_60_days',
    'CPC_last_60_days',
    'CPA_last_60_days',
    'ACOS_last_60_days',
    'CVR_last_60_days',
    'CTR_last_60_days',

    # ---- Monthly Metrics ----
    'impressions_Sep',
    'clicks_Sep',
    'ad_spend_Sep',
    'ads_sale_Sep',
    'ads_units_Sep',
    'CPC_Sep',
    'CPA_Sep',
    'ACOS_Sep',
    'CVR_Sep',
    'CTR_Sep',

    'impressions_Oct',
    'clicks_Oct',
    'ad_spend_Oct',
    'ads_sale_Oct',
    'ads_units_Oct',
    'CPC_Oct',
    'CPA_Oct',
    'ACOS_Oct',
    'CVR_Oct',
    'CTR_Oct',

    'impressions_Nov',
    'clicks_Nov',
    'ad_spend_Nov',
    'ads_sale_Nov',
    'ads_units_Nov',
    'CPC_Nov',
    'CPA_Nov',
    'ACOS_Nov',
    'CVR_Nov',
    'CTR_Nov',

    # ---- Summary ----
    'cumulative_sales',
    'total_sales_portfolio',
    'cumulative_percent',
]
df=df.select(columns_adjusted)
df.write \
    .format("jdbc") \
    .option("url", URL) \
    .option("user", USER) \
    .option("password", PWD) \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "public.kt_pt_ct_report") \
    .mode("overwrite") \
    .save()



25/11/09 00:36:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/09 00:39:27 WARN DAGScheduler: Broadcasting large task binary with size 1126.1 KiB
25/11/09 00:39:33 WARN DAGScheduler: Broadcasting large task binary with size 1104.1 KiB


In [4]:


ads_automation_sheet='14aA8eKlsJqCd8mrNZcZOcnwW5pGICTZKs7DVE-IjO8Q'
rules=read_sheet(ads_automation_sheet,'Rules',2)
from sqlalchemy import (
    create_engine,

    text,

)
connection_string = (
    f"postgresql://{user}:{password}@{ip}:{5433}/{db}"
)

def query_to_df(db_url,sql_query):
    db_url = db_url

    engine = create_engine(db_url)
    with engine.connect() as connection:
        result = connection.execute(text(sql_query))
        
        # Fetch the results
        rows = result.fetchall()
        
        df=pd.DataFrame(rows)
    return df
query=""" 
select * from kt_pt_ct_report where account_name = 'awenest'
"""
df=query_to_df(connection_string,query)
df["ACOS_last_14_days"] = df["ACOS_last_14_days"].astype(float)
df['Acos_ratio']= df['ACOS_last_14_days'].div(df['target_acos']).astype("float").round(2)
account_rules=rules[rules['account_name']=='awenest']

rulesdf = account_rules.replace("", pd.NA)
end_cols=[col for col in rulesdf.columns if 'end' in col]
all_nums=[]
for each_col in end_cols:
    num=int(each_col.split("_")[0])
    all_nums.append(num)

In [5]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

# --- Fetch data from Postgres with column names ---
def query_to_df(db_url, sql_query):
    engine = create_engine(db_url)
    with engine.connect() as connection:
        result = connection.execute(text(sql_query))
        rows = result.fetchall()
        cols = result.keys()              # ✅ fetch column names
        df = pd.DataFrame(rows, columns=cols)
    return df


# --- Load data ---
query = """select * from kt_pt_ct_report where account_name = 'awenest'"""
df = query_to_df(connection_string, query)
import decimal

# Convert all Decimal columns to float
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, decimal.Decimal)).any():
        df[col] = df[col].astype(float)

df['Acos_ratio'] = df['ACOS_last_14_days'].div(df['target_acos']).astype(float).round(2)

# --- Prepare rules ---
account_rules = rules[rules['account_name'] == 'awenest']
rulesdf = account_rules.replace("", pd.NA).copy()

end_cols = [c for c in rulesdf.columns if c.endswith("_end")]
all_nums = [int(c.split("_")[0]) for c in end_cols]
max_col_index = max(all_nums)

df["suggested_bid"] = np.nan
df["_portfolio_lower"] = df["portfolio_name"].fillna("").str.strip().str.lower()

# --- Apply all rules sequentially ---
for idx, rule in rulesdf.iterrows():
    mask = pd.Series(True, index=df.index)

    # 1️⃣ Portfolio filter
    portfolios_raw = rule.get("portfolio_name")
    if pd.notna(portfolios_raw) and str(portfolios_raw).strip():
        allowed = [p.strip().lower() for p in str(portfolios_raw).split(",") if p.strip()]
        mask &= df["_portfolio_lower"].isin(allowed)
    # else blank = all portfolios

    # 2️⃣ Column filters
    for i in range(1, max_col_index + 1):
        col_filter = rule.get(f"{i}_column_filter")
        start = rule.get(f"{i}_start")
        end = rule.get(f"{i}_end")

        # Skip blank sets
        if pd.isna(col_filter) and pd.isna(start) and pd.isna(end):
            continue
        if pd.isna(col_filter) or col_filter not in df.columns:
            continue

        df[col_filter] = pd.to_numeric(df[col_filter], errors="coerce").fillna(0)

        if pd.notna(start) and pd.notna(end):
            mask &= (df[col_filter] >= float(start)) & (df[col_filter] < float(end))
        elif pd.notna(start):
            mask &= (df[col_filter] >= float(start))
        elif pd.notna(end):
            mask &= (df[col_filter] < float(end))

    if not mask.any():
        print(f"❌ Rule {idx+1}: no matches → skipped")
        continue

    cpc_col = rule["cpc_column_name"]
    bid_change = str(rule["bid_change"]).strip().lower()
    if cpc_col not in df.columns:
        print(f"⚠️ Rule {idx+1}: CPC column '{cpc_col}' not found → skipped")
        continue

    target_mask = mask & df["suggested_bid"].isna()   # only update empty bids

    if bid_change == "no change":
        df.loc[target_mask, "suggested_bid"] = df.loc[target_mask, cpc_col]

    elif bid_change == "cpc/ratio":
        df.loc[target_mask, "suggested_bid"] = np.where(
            df.loc[target_mask, "Acos_ratio"] > 0,
            df.loc[target_mask, cpc_col] / df.loc[target_mask, "Acos_ratio"].replace(0, np.nan),
            df.loc[target_mask, cpc_col]
        )

    else:
        try:
            pct = float(bid_change)
            factor = 1 + (pct / 100.0)
            df.loc[target_mask, "suggested_bid"] = df.loc[target_mask, cpc_col].astype(float) * factor
        except ValueError:
            continue

    print(f"✅ Rule {idx+1}: updated {target_mask.sum()} rows → bid_change='{bid_change}'")

# --- Cleanup ---
df.drop(columns=["_portfolio_lower"], inplace=True)
print(f"\nTotal rows with suggested_bid: {df['suggested_bid'].notna().sum()} / {len(df)}")


✅ Rule 1: updated 2137 rows → bid_change='10'
✅ Rule 2: updated 1576 rows → bid_change='10'
✅ Rule 3: updated 55 rows → bid_change='5'
✅ Rule 4: updated 40 rows → bid_change='2'
✅ Rule 5: updated 0 rows → bid_change='-50'
✅ Rule 6: updated 0 rows → bid_change='10'
✅ Rule 7: updated 0 rows → bid_change='no change'
✅ Rule 8: updated 184 rows → bid_change='cpc/ratio'

Total rows with suggested_bid: 3992 / 3992


In [6]:
def print_sheet(index,final_result, sheet_name, sheet_id, x, y, mode=1,make_upper_case=0):
    from datetime import datetime
    import time
    current_time = datetime.now()
    current_date_string = current_time.strftime('%Y-%m-%d')
    if "updated_date" not in final_result.columns:
        final_result['updated_date'] = current_date_string
    index=int(index)
    index=index%12
    # time.sleep(4)
    service_file_path = os.path.join(os.getcwd(), f"key{index}.json")
    print(service_file_path)
    gc = pygsheets.authorize(service_file=service_file_path)
    sh=gc.open_by_key(sheet_id)
    selected_sheet = sh.worksheet_by_title(sheet_name)
    if make_upper_case==1:
        column_names = final_result.columns.tolist()
        converted_columns = []
        for column_name in column_names:
            column_name = column_name.split('_')
            column_name = list(map(lambda x:x.capitalize(), column_name))
            column_name = (' ').join(column_name)
            converted_columns.append(column_name)
        names_mapping = dict(zip(column_names, converted_columns))
        final_result = final_result.rename(columns=names_mapping)

    

    """ 
    Mode 2 will now append data while keeping the headers intact and delete the column names row for the new data.
    Mode 2 adds only 2 new rows
    """
    """
    Mode 3 will also add data while maintaining headers, but will delete the column names row for the new data.
    Mode 3 adds the entire lenght of the new dataframe + 2 rows
    """
    if mode == 1:
        selected_sheet.set_dataframe(final_result, (x, y), fit=True)

    elif mode == 2:
        last_filled_row = len(selected_sheet.get_as_df())
        selected_sheet.add_rows(2)
        print_from = last_filled_row + 2

        if last_filled_row == 0:
            selected_sheet.set_dataframe(final_result, (1, 1))

        else:
            selected_sheet.set_dataframe(final_result, (print_from, 1))
            selected_sheet.delete_rows(print_from)


    elif mode == 3:
        last_filled_row = len(selected_sheet.get_as_df())
        total_rows_needed = last_filled_row + len(final_result) + 2
        existing_rows = selected_sheet.rows

        # Add rows only if needed
        if total_rows_needed > existing_rows:
            selected_sheet.add_rows(total_rows_needed - existing_rows)

        time.sleep(3)
        print_from = last_filled_row + 2 if last_filled_row > 0 else 1
        selected_sheet.set_dataframe(final_result, (print_from, 1))

        if last_filled_row > 0:
            selected_sheet.delete_rows(print_from)
    else:
        selected_sheet.set_dataframe(final_result, (x, y))
    print(f"uploaded * sheet name *{sheet_name}* in sheet_id *{sheet_id}")




In [7]:
df['keyword']=df['keyword'].str.replace("+","*")
df['suggested_bid']=df['suggested_bid'].round(1)
suggested_bid=df['suggested_bid']
df=df.drop("suggested_bid",axis=1)
df.insert(14,"suggested_bid",suggested_bid)
df['Acos_ratio']=df['Acos_ratio'].round(1)
Acos_ratio=df['Acos_ratio']
df=df.drop("Acos_ratio",axis=1)
df.insert(15,"Acos_ratio",Acos_ratio)
df=df.sort_values(by='ad_spend_last_14_days',ascending=False)
print_sheet(1,df, 'kt_pt_ct_report', '1VHNcEG4EExsROyhb4eTU0yl4JYktNOrBW9VeUCU25cw', x=1, y=1, mode=1,make_upper_case=0)

/tmp/ipykernel_1359660/4200566825.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_result['updated_date'] = current_date_string


/home/jovyan/work/data-pipeline-s3-postgres/src/key1.json
uploaded * sheet name *kt_pt_ct_report* in sheet_id *1VHNcEG4EExsROyhb4eTU0yl4JYktNOrBW9VeUCU25cw
